In [ ]:
!pip install Keras-Preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
!wget https://www.dropbox.com/scl/fo/u6cxdy4afbot71qlx1htf/h?rlkey=qra18nhi1jpk2m7r0vv1umr9s&dl=0

--2024-10-14 06:47:55--  https://www.dropbox.com/scl/fo/u6cxdy4afbot71qlx1htf/h?rlkey=qra18nhi1jpk2m7r0vv1umr9s
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd1a221e77af6cb959e516d0cbc.dl.dropboxusercontent.com/zip_download_get/B_wm5uDYijHLfCqZCB9UXUqpY_Q5YP5Rqwnx1kecau2UaMwB7wNAeK5eMkzZ3lDmo1sEckAfjCZNbtmX6ctnUrwfVdcJhEDaCz3wx2_BiLUaxg# [following]
--2024-10-14 06:48:01--  https://ucd1a221e77af6cb959e516d0cbc.dl.dropboxusercontent.com/zip_download_get/B_wm5uDYijHLfCqZCB9UXUqpY_Q5YP5Rqwnx1kecau2UaMwB7wNAeK5eMkzZ3lDmo1sEckAfjCZNbtmX6ctnUrwfVdcJhEDaCz3wx2_BiLUaxg
Resolving ucd1a221e77af6cb959e516d0cbc.dl.dropboxusercontent.com (ucd1a221e77af6cb959e516d0cbc.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d:10f
Connecting to ucd1a221e77af6cb959e516d0cbc.dl.dropboxusercontent.com (ucd1

In [ ]:
!unzip h?rlkey=qra18nhi1jpk2m7r0vv1umr9s

Streaming output truncated to the last 5000 lines.
 extracting: Flickr_Data/Flickr_Data/Images/3435015880_eda46ff50f.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3475005101_6f6e437459.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3479050296_65bcea69a0.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3486538055_bcf4d3dfda.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3494105596_f05cb0d56f.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3500136982_bf7a85531e.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3521201948_9049197f20.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3529211822_1dabdb3a9c.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3532205154_5674b628ea.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3532587748_7e64bb223a.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3540598210_972f0ff573.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3547000169_40191e02ca.jpg  
 extracting: Flickr_Data/Flickr_Data/Images/3549011001_26cace3646.jpg  
 extracting: 

In [ ]:
####IMPORTS###
import numpy as np
import tensorflow.keras as keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import collections
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.utils import plot_model
import nltk
from nltk.corpus import stopwords
import pickle
from keras.models import Model, load_model
from keras.layers import Input,Dense, Dropout, Embedding, LSTM
from keras.utils import to_categorical

In [ ]:
def read_text(path):
  with open(path,'r') as f:
    captions = f.read()
  return captions

In [ ]:
train_path = "/content/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt"
test_path = "/content/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt"

In [ ]:
train_data = read_text(train_path)
test_data = read_text(test_path)

In [ ]:
train_data = train_data.split('\n')
test_data = test_data.split('\n')
train_data = train_data[:-1] # IMAGES KEPT FOR TRAINING
test_data = test_data[:-1] # IMAGES KEPT FOR TESTING

In [ ]:
model = tf.keras.applications.InceptionV3(weights = "imagenet", input_shape = (224,224,3), include_top = False)
model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 111, 111, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 111, 111, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 111, 111, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 109, 109, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 109, 109, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 109, 109, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 109, 109, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 109, 109, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 109, 109, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 54, 54, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 54, 54, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 54, 54, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 54, 54, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 52, 52, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 52, 52, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [ ]:
def pre_processing(img):
  img = image.load_img(img,target_size = (224,224,3))
  img = image.img_to_array(img)
  img = np.expand_dims(img,axis = 0)
  img = preprocess_input(img)
  return img

In [ ]:
def encode_image(img):
  img = pre_processing(img)
  feature_vec = model.predict(img)
  feature_vec = feature_vec.reshape((-1,))
  return feature_vec

In [ ]:
save_path = "/content/drive/MyDrive/Images_Encoded/"
ori_img_path = "/content/Flickr_Data/Flickr_Data/Images/"

In [ ]:
train_encoded = {}
n = len(train_data)
for ix,img_id in enumerate(train_data):
  img_path = ori_img_path+img_id
  print(ix,"/",n)
  train_encoded[img_id] = encode_image(img_path)

Streaming output truncated to the last 5000 lines.
3500 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
3501 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
3502 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
3503 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3504 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3505 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
3506 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
3507 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
3508 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
3509 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
3510 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
3511 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
3512 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
3513 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
3514 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
3515 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
3516 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
3517 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
3518 / 6000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

In [ ]:
test_encoded = {}
n = len(test_data)
for ix,img_id in enumerate(test_data):
  img_path = ori_img_path+img_id
  print(ix,"/",n)
  test_encoded[img_id] = encode_image(img_path)

0 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
3 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
5 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
6 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
7 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
8 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
10 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
11 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
12 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
13 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
14 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
15 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
16 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
17 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
18 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
19 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
20 / 1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
21

In [ ]:
import pickle
with open("/content/drive/MyDrive/Images_Encoded/encoded_training_features_VGG.pkl","wb") as f:
  pickle.dump(train_encoded,f)
with open("/content/drive/MyDrive/Images_Encoded/encoded_testing_features_VGG.pkl","wb") as f:
  pickle.dump(test_encoded,f)